# Big Reaction Network Visualization Sandbox

### Import dependencies

In [1]:
using Catalyst
using Graphs
using LinearAlgebra

using GraphPlot
using Plots

# load test reaction network
include("../data/mcm_3-2.jl");

In [2]:
## Takes too long
# Graph(test_rn)

In [3]:
# reactionparams(test_rn)
nspecs = numspecies(test_rn)

5733

In [4]:
numreactions(test_rn)

16404

In [5]:
numreactionparams(test_rn)

0

In [6]:
specs = species(test_rn);

### Reaction Complexes

In [7]:
# smap=speciesmap(test_rn);
reactioncomplexmap(test_rn);

In [8]:
# vector of rxn complexes and an incidence matrix (maps complexes to reactions)
v, B = reactioncomplexes(test_rn; sparse=true);

()

In [9]:
# v[1][1].speciesstoich

v[1].speciesids

1-element Vector{Int64}:
 1

In [10]:
# matrix mapping species to reaction complexes
s2c_mat = complexstoichmat(test_rn; sparse=true);

In [11]:
# matrix mapping species to reactions
s2r_mat = complexoutgoingmat(test_rn; sparse=true);

In [12]:
# sir = @reaction_network SIR begin
#     β, S + I --> 2I
#     ν, I --> R
# end β ν
# rcs,incidencemat = reactioncomplexes(sir)
# incidencegraph   = incidencematgraph(incidencemat)

# gplot(incidencegraph);

In [13]:
# # top substrates
# minval, minindx = findmin(sum(B, dims=2))
# println("Most prevalent substrate: ", string(specs[v[minindx].speciesids][1]))

# # # top products
# maxval, maxindx = findmax(sum(B, dims=2))
# println("Most prevalent product: ", string(specs[v[maxindx].speciesids][1]))

In [14]:
isorted = sortperm(sum(B, dims=2)[:,1])
n = 7

# top n substrates
println("Top ",string(n)," substrates: ")
for i = 1:n
    println(" - ",string(specs[v[isorted[i]].speciesids][1]))
end
println("")
# top n products
numComplexes = size(B)[1]
println("Top ",string(n)," products: ")
for i = 1:n
    println(" - ",string(specs[v[isorted[numComplexes+1-i]].speciesids][1]))
end

Top 7 substrates: 
 - TM123BPRO(t)
 - OH(t)
 - OH(t)
 - OH(t)
 - CL(t)
 - OH(t)
 - OH(t)

Top 7 products: 
 - OH(t)
 - NO2(t)
 - NO2(t)
 - NO2(t)
 - NO2(t)
 - CO(t)
 - HO2(t)


In [15]:
# g = complexgraph(test_rn);
# savegraph(g, "complexnet")

### Graphs

In [16]:
# # create "random" adjacency matrix
# threshold = 0.5; # define threshold
# A = vcat(hcat(10*rand(4,4), rand(4,4)), hcat(rand(4,4), rand(4,4)));
# # A = rand(8,8)
# A = A - 10*I(8);
# A = round.(Int, A)
# A = A'*A;
# A[findall(A .< threshold)] .= 0;
# A[findall(A .> threshold)] .= 1;
# A = A - I(8);

In [17]:
# # create graph using Graph.jl
# G = SimpleGraph(A)
# gplot(G)

In [18]:
# function to create adjacency matrix such that nodes are species and edges exist between species if they appear together in a reaction
function mkAdjMat(test_rn)

    nspecs = numspecies(test_rn)
    specs = species(test_rn);

    # initialize adjacency matrix
    A = convert.(Int, zeros(nspecs,nspecs));

    for rxn = reactions(test_rn)
        # define vector with all species
        rxn_species = vcat(rxn.substrates, rxn.products)

        # intialize vector to store indicies of species that appear in current reaction
        rxn_species_indexes = []
        # loop through species
        for spec in rxn_species
            # find corresponding index for current species
            rxn_species_indexes = vcat(rxn_species_indexes, findall(x -> x==string(spec), string.(specs)))
        end

        # loop through indicies and update adajcency matrix such that an edge is added between all species in reaction
        for i in rxn_species_indexes
            for j in rxn_species_indexes
                if i==j
                    continue
                end
                if j<i
                    continue
                end

                A[i,j] =  A[i,j] + 1;
                A[j,i] =  A[j,i] + 1;
            end
            
            if i ==100 
                break
            end
        end
        
        # print(rxn.species)
        
    end
    
    return A
end;

In [19]:
@time A = mkAdjMat(test_rn)

433.674505 seconds (5.09 G allocations: 201.079 GiB, 3.04% gc time, 0.07% compilation time)


5733×5733 Matrix{Int64}:
 0    3     3     5     2  1  0     0  …  0  0  0  0  0  0  0  0  0  0  0  0
 3    0     1     2     2  1  0    10     0  0  0  0  0  0  0  0  0  0  0  0
 3    1     0  1257     3  0  0     6     0  0  0  1  1  2  0  0  0  0  0  1
 5    2  1257     0  1259  0  2   743     0  1  0  4  4  4  0  3  0  3  0  2
 2    2     3  1259     0  0  2    17     0  0  1  1  2  2  0  0  0  0  0  1
 1    1     0     0     0  0  0     1  …  0  0  0  0  0  0  0  0  0  0  0  0
 0    0     0     2     2  0  0     0     0  0  0  0  0  0  0  0  0  0  0  0
 0   10     6   743    17  1  0     0     1  4  2  3  3  3  2  1  2  1  2  1
 0  178    53   194    57  0  0  1066     1  2  2  2  2  3  1  0  1  0  1  0
 0    0     0     0     0  0  0     1     0  0  0  0  0  0  0  0  0  0  0  0
 0    6    21   343    53  0  0   510  …  0  2  1  0  0  1  0  1  0  0  0  1
 0    0     0     0     0  0  0     2     0  0  0  0  0  0  0  0  0  0  0  0
 0    0     2     1     0  0  0     3     0  0  0  

In [20]:
# A[1:10,1:10]

In [21]:
# create graph using Graph.jl
G = SimpleGraph(A)

{5733, 35050} undirected simple Int64 graph

In [22]:
# # plot graph
# @time gplot(G)

### Centrality

In [23]:
@time bet_cen = betweenness_centrality(G);

# plot(bet_cen)

isorted = sortperm(bet_cen)
n = 7
# top n species
println("Top ",string(n)," species based on betweenness centrality: ")
for i = 1:n
    println(" - ",string(specs[v[isorted[i]].speciesids][1]))
end

  5.729687 seconds (59.88 M allocations: 9.036 GiB, 10.23% gc time, 2.22% compilation time)
Top 7 species based on betweenness centrality: 
 - O1D(t)
 - NO(t)
 - NO(t)
 - O3(t)
 - TC4H9O(t)
 - BUTALO2(t)
 - OH(t)


In [24]:
@time deg_cen = degree_centrality(G);

# plot(deg_cen);

isorted = sortperm(deg_cen)
n = 7
# top n species
println("Top ",string(n)," species based on degree centrality: ")
for i = 1:n
    println(" - ",string(specs[v[isorted[i]].speciesids][1]))
end

  0.010541 seconds (22.65 k allocations: 1.295 MiB, 99.68% compilation time)
Top 7 species based on degree centrality: 
 - O3(t)
 - IPEBOOH(t)
 - OH(t)
 - CH3CHO(t)
 - OH(t)
 - NO3(t)
 - OH(t)


In [25]:
@time eig_cen = eigenvector_centrality(G);

isorted = sortperm(eig_cen)
n = 7
# top n species
println("Top ",string(n)," species based on eigenvector centrality: ")
for i = 1:n
    println(" - ",string(specs[v[isorted[i]].speciesids][1]))
end

  2.832283 seconds (18.65 M allocations: 918.957 MiB, 4.12% gc time, 99.87% compilation time)
Top 7 species based on eigenvector centrality: 
 - IPEBOOH(t)
 - O3(t)
 - OH(t)
 - CL(t)
 - NO(t)
 - OH(t)
 - OH(t)


In [26]:
@time kat_cen = katz_centrality(G, 0.3);

isorted = sortperm(kat_cen)
n = 7
# top n species
println("Top ",string(n)," species based on katz centrality: ")
for i = 1:n
    println(" - ",string(specs[v[isorted[i]].speciesids][1]))
end

  0.947263 seconds (4.96 M allocations: 276.464 MiB, 98.80% compilation time)
Top 7 species based on katz centrality: 
 - OH(t)
 - NO2(t)
 - HO2(t)
 - NO3(t)
 - NO(t)
 - MPRKBOOH(t)
 - EGLYOX(t)


In [27]:
# # visualize sparse network
# https://juliagraphs.org/Graphs.jl/dev/first_steps/plotting/
# https://fcdimitr.github.io/SGtSNEpi.jl/stable/

# # example visualizing large sparse network
# GLMakie.activate!()

# # error with imports!
# using GLMakie
# using SGtSNEpi
# using SNAPDatasets

# g = loadsnap(:as_caida)
# y = sgtsnepi(g);
# show_embedding(y;
#   A = adjacency_matrix(g),        # show edges on embedding
#   mrk_size = 1,                   # control node sizes
#   lwd_in = 0.01, lwd_out = 0.001, # control edge widths
#   edge_alpha = 0.03 )             # control edge transparency

### Community Detection

In [32]:
@time cliqPer_output = clique_percolation(G, k=14);

specs[Int.(cliqPer_output[1])]

  0.576849 seconds (481.39 k allocations: 50.903 MiB, 6.65% gc time)


15-element Vector{Any}:
 O3(t)
 NO(t)
 NO2(t)
 NO3(t)
 OH(t)
 HO2(t)
 CO(t)
 HCHO(t)
 CH3CHO(t)
 C2H5CHO(t)
 CH3COCH3(t)
 CH3CO3(t)
 C2H5CO3(t)
 GLYOX(t)
 MGLYOX(t)

In [33]:
@time corePeri_output = core_periphery_deg(G)

println("Core species:")
specs[corePeri_output.==1]

  0.000372 seconds (8 allocations: 134.984 KiB)
Core species:


41-element Vector{Any}:
 O3(t)
 NO(t)
 NO2(t)
 NO3(t)
 OH(t)
 HO2(t)
 CO(t)
 H2O2(t)
 HNO3(t)
 SO2(t)
 SO3(t)
 HCHO(t)
 CH3CHO(t)
 ⋮
 CO3C4CHO(t)
 CO2H3CHO(t)
 EPXC4DIAL(t)
 BIACET(t)
 EGLYOX(t)
 HOC2H4CO3(t)
 HCOCO2H(t)
 C5DICARB(t)
 C33CO(t)
 CH3COCO2H(t)
 HCOCH2CO3(t)
 HCOCH2CHO(t)

In [34]:
# rich club coefficient
# from wiki: designed to measure the extent to which well-connected nodes also connect to each other. 
# Networks which have a relatively high rich-club coefficient are said to demonstrate the rich-club effect and will have 
# many connections between nodes of high degree. 

@time rich_club(G, 100)

  0.000281 seconds (6 allocations: 49.891 KiB)


0.9019607843137255

### Construct directed graph

### Cycle Detection

In [36]:
@time cyc_basis = cycle_basis(G, nothing)

  0.159897 seconds (545.20 k allocations: 43.762 MiB, 79.47% compilation time)


29318-element Vector{Vector{Int64}}:
 [8, 5224, 4986]
 [9, 5224, 4986]
 [4, 5386, 5224, 4986]
 [8, 5386, 5224]
 [9, 5386, 5224]
 [32, 5386, 5224]
 [5221, 5386, 5224, 4986]
 [5222, 5386, 5224, 4986]
 [8, 5500, 5386]
 [9, 5500, 5386]
 [5387, 5500, 5386]
 [8, 5498, 5386]
 [9, 5498, 5386]
 ⋮
 [5, 9, 4986]
 [8, 9, 4986]
 [4, 15, 9]
 [8, 15, 9]
 [3, 8, 4986]
 [4, 8, 4986]
 [5, 8, 4986]
 [3, 13, 8]
 [4, 13, 8]
 [3, 5, 4986]
 [4, 5, 4986]
 [3, 4, 4986]